In [1]:
import requests
url = "https://dhlottery.co.kr/common.do?method=main" 
html = requests.get(url).text

In [2]:
import requests 
from bs4 import BeautifulSoup 
def getMaxRoundNum() -> int: 
    url = "https://dhlottery.co.kr/common.do?method=main"
    html = requests.get(url).text 
    soup = BeautifulSoup(html, "lxml") 
    tag = soup.find(name="strong", attrs={"id": "lottoDrwNo"}) 
    return int(tag.text)

In [3]:
getMaxRoundNum()

962

In [4]:
import requests 
from datetime import datetime 
from bs4 import BeautifulSoup 
def getWinNumbers(round_num: int): 
    url = f"https://dhlottery.co.kr/gameResult.do?method=byWin&drwNo={round_num}"
    html = requests.get(url).text 
    soup = BeautifulSoup(html, "lxml") 
    win_result_tag = soup.find(name="div", attrs={"class": "win_result"}) 
    strong_tags = win_result_tag.find_all("strong")
    round_num_text = strong_tags[0].text.replace("회", '') 
    round_num_query = int(round_num_text) # 추첨일 읽기
    p_tags = win_result_tag.find_all("p", "desc") 
    draw_date = datetime.strptime(p_tags[0].text, "(%Y년 %m월 %d일 추첨)") # 당첨번호 6개 읽기 
    num_win_tag = win_result_tag.find(name="div", attrs={"class": "num win"}) 
    p_tag = num_win_tag.find("p") 
    win_nums = [int(x.text) for x in p_tag.find_all("span")] # 보너스 번호 읽기
    num_bonus_tag = win_result_tag.find(name="div", attrs={"class": "num bonus"}) 
    p_tag = num_bonus_tag.find("p") 
    bonus_num = int(p_tag.find("span").text)
    return { "round_num": round_num_query, "draw_date": draw_date, "win_nums": win_nums, "bonus_num": bonus_num }

In [6]:
lottery_dict = [getWinNumbers(num) for num in  range(1,getMaxRoundNum()+1)]

In [9]:
import pandas as pd
data = pd.DataFrame(lottery_dict)

In [11]:
data

,round_num,draw_date,win_nums,bonus_num
0,1,2002-12-07,"[10, 23, 29, 33, 37, 40]",16
1,2,2002-12-14,"[9, 13, 21, 25, 32, 42]",2
2,3,2002-12-21,"[11, 16, 19, 21, 27, 31]",30
3,4,2002-12-28,"[14, 27, 30, 31, 40, 42]",2
4,5,2003-01-04,"[16, 24, 29, 40, 41, 42]",3
...,...,...,...,...
957,958,2021-04-10,"[2, 9, 10, 16, 35, 37]",1
958,959,2021-04-17,"[1, 14, 15, 24, 40, 41]",35
959,960,2021-04-24,"[2, 18, 24, 30, 32, 45]",14
960,961,2021-05-01,"[11, 20, 29, 31, 33, 42]",43


In [10]:
data.to_csv("./dhlottery_data.csv",index=False)

In [10]:
import pymysql 
db = pymysql.connect(host="localhost",port=8889,user="root",passwd="tabasco3204-1")


KeyboardInterrupt: 

In [ ]:
import pymysql 
def connect_db(): 
    conn = pymysql.connect( host='MySQL Address', port=MySqlPort, 
    user='MySQL Account', passwd='MySQL Account Password', db='mydata' ) 
    return conn 
def getTableSchema(): 
    db = connect_db() 
    cursor = db.cursor() 
    cursor.execute("DESCRIBE LOTTO;") 
    result = cursor.fetchall() 
    db.close() 
    return result

def update_db():
    db = connect_db() 
    cursor = db.cursor() # 현재 테이블에 등록된 회차 쿼리
    cursor.execute("SELECT ROUND FROM LOTTO;") 
    result = cursor.fetchall() 
    round_num_in_table = [x[0] for x in result] if len(round_num_in_table) > 0: print("Already {} records are updated".format(len(round_num_in_table))) # 최대 회차 가져오기 
    max_round = getMaxRoundNum() # 테이블 업데이트 
    for r in range(1, max_round + 1): # 이미 등록된 레코드인지 체크 
        if r in round_num_in_table: 
            continue 
        print(f"Get Lotto Win Number (Round: {r})") 
        crawl_result = getWinNumbers(r) 
        sql = "INSERT INTO LOTTO (`ROUND`, `DATE`, NUM_1, NUM_2, NUM_3, NUM_4, NUM_5, NUM_6, NUM_BONUS) "
        sql += "VALUES ({}, '{}', {}, {}, {}, {}, {}, {}, {});".\
            format( r, crawl_result["draw_date"].strftime("%Y-%m-%d"), 
             crawl_resul["win_nums"][0], crawl_result["win_nums"][1], 
             crawl_result["win_nums"][2],crawl_result["win_nums"][3],
             crawl_result["win_nums"][4], crawl_resul["win_nums"][5], 
             crawl_result["bonus_num"] ) 
        
        result = cursor.execute(sql)
        print(">> Update query result: {}".format(result)) 
        db.commit();db.close()


In [12]:
import pandas as pd 
def get_data_from_db(): 
    db = connect_db() 
    cursor = db.cursor() # 테이블 Column 이름 얻어오기 
    cursor.execute(f"SHOW columns FROM LOTTO;") 
    columns = [tuple(x)[0] for x in cursor.fetchall()]
    # 테이블 모든 레코드 가져오기(SELECT) 
    cursor.execute("SELECT * FROM LOTTO;") 
    df = pd.DataFrame(list(cursor.fetchall())) 
    df.columns = columns 
    db.close() 
    return df